In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [4]:
df['Initial'] = 0
for i in df:
    df['Initial'] = df.Name.str.extract('([A-Za-z]+)\.')

In [5]:
df['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [6]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Initial          0
dtype: int64

In [7]:
#Assigning the NaN values with Ceil values of the mean ages
df.loc[(df.Age.isnull())&(df.Initial=='Mr'),'Age']=33
df.loc[(df.Age.isnull())&(df.Initial=='Mrs'),'Age']=36
df.loc[(df.Age.isnull())&(df.Initial=='Master'),'Age']=5
df.loc[(df.Age.isnull())&(df.Initial=='Miss'),'Age']=22
df.loc[(df.Age.isnull())&(df.Initial=='Other'),'Age']=46         

In [8]:
df['Embarked'].fillna('S',inplace=True)

In [9]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Initial
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss


In [10]:
df['Cabin'] = df['Cabin'].apply(lambda x:not isinstance(x,float))

In [11]:
df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

In [12]:
label_encoder = {}

for x in ['Sex','Embarked','Initial']:
    label_encoder[x] = LabelEncoder()
    df[x] = label_encoder[x].fit_transform(df[x])

In [13]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Initial
0,0,3,1,22.0,1,0,7.2500,False,2,2
1,1,1,0,38.0,1,0,71.2833,True,0,3
2,1,3,0,26.0,0,0,7.9250,False,2,1


In [14]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Cabin       0
Embarked    0
Initial     0
dtype: int64

In [15]:
X = df[df.columns[1:]].values
y = df['Survived']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [17]:
def scores(y_true, y_pred):
    results = [accuracy_score(y_true, y_pred),
              precision_score(y_true, y_pred),
              recall_score(y_true, y_pred),
               f1_score(y_true, y_pred),
               roc_auc_score(y_true, y_pred)
              ]
    return [round(x, 2) for x in results]

In [18]:
def create_models(X_train, X_test, y_train, y_test):
    print('Logistic Regression')
    model = LogisticRegressionCV()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = scores(y_test, y_pred)
    print('Accuracy: {}, Precision: {}, Recall: {}, ROC_AUC: {}'.format(
    results[0], results[1], results[2], results[3],) ) 
    
    print('\n')
    
    print('Decision Tree')
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = scores(y_test, y_pred)
    print('Accuracy: {}, Precision: {}, Recall: {}, ROC_AUC: {}'.format(
    results[0], results[1], results[2], results[3],) ) 
     
    print('\n')
    
    print('SVM')
    model = SVC()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = scores(y_test, y_pred)
    print('Accuracy: {}, Precision: {}, Recall: {}, ROC_AUC: {}'.format(
    results[0], results[1], results[2], results[3],) ) 
    
    print('\n')

    print('Random Forest')
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results = scores(y_test, y_pred)
    print('Accuracy: {}, Precision: {}, Recall: {}, ROC_AUC: {}'.format(
    results[0], results[1], results[2], results[3],) ) 

In [19]:
create_models(X_train, X_test, y_train, y_test)

Logistic Regression
Accuracy: 0.82, Precision: 0.8, Recall: 0.76, ROC_AUC: 0.78


Decision Tree
Accuracy: 0.82, Precision: 0.78, Recall: 0.77, ROC_AUC: 0.78


SVM
Accuracy: 0.66, Precision: 0.76, Recall: 0.26, ROC_AUC: 0.38


Random Forest
Accuracy: 0.82, Precision: 0.78, Recall: 0.78, ROC_AUC: 0.78


In [20]:
print('Logistic Regression')
lr = LogisticRegressionCV(random_state=42)
score1 = cross_validate(lr, X,y, cv=10, n_jobs=-1, verbose=1)
print('Score: ',score1['test_score'].mean())

print('\n')

print('Decision Tree')
dt = DecisionTreeClassifier(random_state=42)
score2 = cross_validate(dt, X,y, cv=10, n_jobs=-1, verbose=1)
print('Score: ',score2['test_score'].mean())
     
print('\n')
    
print('SVM')
svm = SVC(random_state=42)
score3 = cross_validate(svm, X,y, cv=10, n_jobs=-1, verbose=1)
print('Score: ',score3['test_score'].mean())
    
print('\n')

print('Random Forest')
rf = RandomForestClassifier(random_state=42)
score4 = cross_validate(rf, X,y, cv=10, n_jobs=-1, verbose=1)
print('Score: ',score4['test_score'].mean())

Logistic Regression


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

...

/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/amar/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

Score:  0.7889762796504369


Decision Tree
Score:  0.778976279650437


SVM


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Score:  0.6779650436953808


Random Forest
Score:  0.8170911360799001


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished
